<a href="https://colab.research.google.com/github/JohnnySunkel/BlueSky/blob/master/Transfer_Learning_CNN_MobileNet_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math

TRAIN_DATA_DIR = 'drive/My Drive/train'
VALIDATION_DATA_DIR = 'drive/My Drive/validation'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                   rotation_range = 20,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   zoom_range = 0.2)
val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size = (IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle = True,
                                                    seed = 12345,
                                                    class_mode = 'categorical')
validation_generator = val_datagen.flow_from_directory(VALIDATION_DATA_DIR,
                                                       target_size = (IMG_WIDTH, IMG_HEIGHT),
                                                       batch_size = BATCH_SIZE,
                                                       shuffle = False,
                                                       class_mode = 'categorical')

def model_maker():
  base_model = MobileNet(include_top = False,
                         input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
  for layer in base_model.layers[:]:
    layer.trainable = False
  input = Input(shape = (IMG_WIDTH, IMG_HEIGHT, 3))
  custom_model = base_model(input)
  custom_model = GlobalAveragePooling2D()(custom_model)
  custom_model = Dense(64, activation = 'relu')(custom_model)
  custom_model = Dropout(0.5)(custom_model)
  predictions = Dense(NUM_CLASSES, activation = 'softmax')(custom_model)
  return Model(inputs = input, outputs = predictions)

model = model_maker()
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(0.001),
              metrics = ['acc'])
model.fit_generator(train_generator,
                    steps_per_epoch = math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
                    epochs = 10,
                    validation_data = validation_generator,
                    validation_steps = math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

model.save('model.h5')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/10
8/8 [==============================] - 227s 28s/step - loss: 0.7140 - acc: 0.6797 - val_loss: 0.0643 - val_acc: 0.9883
Epoch 2/10
8/8 [==============================] - 95s 12s/step - loss: 0.2943 - acc: 0.8750 - val_loss: 0.0586 - val_acc: 0.9824
Epoch 3/10
8/8 [==============================] - 101s 13s/step - loss: 0.2201 - acc: 0.8945 - val_loss: 0.0184 - val_acc: 0.9961
Epoch 4/10
8/8 [==============================] - 107s 13s/step - loss: 0.1855 - acc: 0.9180 - val_loss: 0.0258 - val_acc: 0.9922
Epoch 5/10
8/8 [==============================] - 101s 13s/step - loss: 0.1321 - acc: 0.9609 - val_loss: 0.0115 - val_acc: 0.9941
Epoch 6/10
8/8 [==============================] - 97s 12s/step - loss: 0.1253 - acc: 0.9526 - val_loss: 0.0307 - val_acc: 0.9922
Epoch 7/10
8/8 [==============================] - 99s 12s/step - loss: 0.1697 - acc: 0.9336 - val_loss: 0.0125 - val_acc: 0.9922
Epoch 8/1